In [1]:
!pip install -q "transformers==4.44.0" "datasets==2.20.0" "peft==0.12.0" "accelerate==0.33.0" "bitsandbytes==0.43.3" "trl==0.9.6" "safetensors==0.4.4"

In [2]:
import torch
import torch.nn as nn
import pandas as pd
from peft import PeftModel, PeftConfig, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Dataset

In [3]:
from datasets import load_dataset

ds = load_dataset("jinya425/Keyphrase_extraction")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


from datasets import load_dataset

ds = load_dataset("zino36/keyword-extraction-dataset")

In [4]:
train_ds = ds['train']
train_ds

Dataset({
    features: ['Abstract', 'Keywords'],
    num_rows: 50
})

In [5]:
def apply_prompt_template(examples):
  content = examples['Abstract']
  keywords = examples['Keywords']


  full_prompt = f"""Below is an instruction that extracts keywords from a paragraph. Write keywords for the paragraph described below.

### Instruction:
Paragraph:
{content}

### Response:
{keywords}
### End
"""
  return { "text": full_prompt }

dataset = train_ds.map(apply_prompt_template)

In [6]:
print(dataset["text"][0])

Below is an instruction that extracts keywords from a paragraph. Write keywords for the paragraph described below.

### Instruction:
Paragraph:
In this work we propose RELDEC, a novel approach for sequential decoding of moderate length low-density parity-check (LDPC) codes. The main idea behind RELDEC is that an optimized decoding policy is subsequently obtained via reinforcement learning based on a Markov decision process (MDP). In contrast to our previous work, where an agent learns to schedule only a single check node (CN) within a group (cluster) of CNs per iteration, in this work we train the agent to schedule all CNs in a cluster, and all clusters in every iteration. That is, in each learning step of RELDEC an agent learns to schedule CN clusters sequentially depending on a reward associated with the outcome of scheduling a particular cluster. We also modify the state space representation of the MDP, enabling RELDEC to be suitable for larger block length LDPC codes than those stu

# Qwen2.5 0.5B

In [7]:
# Load model directly
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B")
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-0.5B",device_map="cuda:0")

tokenizer_config.json:   0%|          | 0.00/7.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/681 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

In [8]:
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((

In [9]:
from peft import LoraConfig, get_peft_model
lora_config = LoraConfig(
    r=64,
    lora_alpha=32,
    target_modules=['o_proj', 'q_proj', 'up_proj', 'v_proj', 'k_proj', 'down_proj', 'gate_proj'],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

In [10]:
import transformers
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    dataset_text_field="text",
    peft_config=lora_config,
    max_seq_length=1500,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        max_steps=50,
        learning_rate=2e-4,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        save_strategy="epoch",
        report_to="tensorboard",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss
1,2.825600
2,2.447100
3,2.659200
4,2.609800
5,2.652000
6,2.303300
7,2.437100
8,2.404000
9,2.631500
10,2.442700


TrainOutput(global_step=50, training_loss=2.1662164402008055, metrics={'train_runtime': 68.4442, 'train_samples_per_second': 2.922, 'train_steps_per_second': 0.731, 'total_flos': 133059712180224.0, 'train_loss': 2.1662164402008055, 'epoch': 4.0})

In [11]:
from transformers import pipeline
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausal

## Inference

In [12]:
prompt = """
Below is an instruction that extracts keywords from a paragraph. Write keywords for the paragraph described below.

### Instruction:
Paragraph:
In this work we propose RELDEC, a novel approach for sequential decoding of moderate length low-density parity-check (LDPC) codes. The main idea behind RELDEC is that an optimized decoding policy is subsequently obtained via reinforcement learning based on a Markov decision process (MDP). In contrast to our previous work, where an agent learns to schedule only a single check node (CN) within a group (cluster) of CNs per iteration, in this work we train the agent to schedule all CNs in a cluster, and all clusters in every iteration. That is, in each learning step of RELDEC an agent learns to schedule CN clusters sequentially depending on a reward associated with the outcome of scheduling a particular cluster. We also modify the state space representation of the MDP, enabling RELDEC to be suitable for larger block length LDPC codes than those studied in our previous work. Furthermore, to address decoding under varying channel conditions, we propose two related schemes, namely, agile meta-RELDEC (AM-RELDEC) and meta-RELDEC (M-RELDEC), both of which employ meta-reinforcement learning. The proposed RELDEC scheme significantly outperforms standard flooding and random sequential decoding for a variety of LDPC codes, including codes designed for 5G new radio.

### Response:
"""


result = pipe(prompt, max_length=1500)
generated_text = result[0]['generated_text']
print(generated_text.split("### End")[0])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=2048) and `max_length`(=1500) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



Below is an instruction that extracts keywords from a paragraph. Write keywords for the paragraph described below.

### Instruction:
Paragraph:
In this work we propose RELDEC, a novel approach for sequential decoding of moderate length low-density parity-check (LDPC) codes. The main idea behind RELDEC is that an optimized decoding policy is subsequently obtained via reinforcement learning based on a Markov decision process (MDP). In contrast to our previous work, where an agent learns to schedule only a single check node (CN) within a group (cluster) of CNs per iteration, in this work we train the agent to schedule all CNs in a cluster, and all clusters in every iteration. That is, in each learning step of RELDEC an agent learns to schedule CN clusters sequentially depending on a reward associated with the outcome of scheduling a particular cluster. We also modify the state space representation of the MDP, enabling RELDEC to be suitable for larger block length LDPC codes than those st

In [13]:
prompt = """
Below is an instruction that extracts keywords from a paragraph. Write keywords for the paragraph described below.

### Instruction:
Paragraph:
This research presents an Attention-aware Deep Network for facial expression recognition in the presence of masks. Recognizing facial expression is a crucial problem in the current scenario due to the widespread use of masks due to the COVID-19 pandemic. The model utilizes attention mechanisms to focus on the relevant facial regions for expression recognition, while disregarding the regions occluded by masks. The model is trained and evaluated on a dataset of individuals wearing masks, and experimental results demonstrate its effectiveness in recognizing facial expressions accurately, even in the presence of masks. The proposed approach significantly improves the performance of expression recognition in comparison to existing methods and shows the potential of attention mechanisms in addressing the challenges of facial expression recognition with masks. Datasets are categorized into 7 Classes which represent 7 human expressions including happy, sad, fear, angry, disgust, neutral and surprised.

### Response:
"""


result = pipe(prompt, max_length=2500)
generated_text = result[0]['generated_text']
print(generated_text.split("### End")[0])

Both `max_new_tokens` (=2048) and `max_length`(=2500) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



Below is an instruction that extracts keywords from a paragraph. Write keywords for the paragraph described below.

### Instruction:
Paragraph:
This research presents an Attention-aware Deep Network for facial expression recognition in the presence of masks. Recognizing facial expression is a crucial problem in the current scenario due to the widespread use of masks due to the COVID-19 pandemic. The model utilizes attention mechanisms to focus on the relevant facial regions for expression recognition, while disregarding the regions occluded by masks. The model is trained and evaluated on a dataset of individuals wearing masks, and experimental results demonstrate its effectiveness in recognizing facial expressions accurately, even in the presence of masks. The proposed approach significantly improves the performance of expression recognition in comparison to existing methods and shows the potential of attention mechanisms in addressing the challenges of facial expression recognition w

# HuggingFaceTB/SmolLM-135M

In [7]:
checkpoint = "HuggingFaceTB/SmolLM-135M"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(checkpoint,device_map="cuda:0", trust_remote_code=True)

In [15]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(49152, 576)
    (layers): ModuleList(
      (0-29): 30 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=576, out_features=576, bias=False)
          (k_proj): Linear(in_features=576, out_features=192, bias=False)
          (v_proj): Linear(in_features=576, out_features=192, bias=False)
          (o_proj): Linear(in_features=576, out_features=576, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=576, out_features=1536, bias=False)
          (up_proj): Linear(in_features=576, out_features=1536, bias=False)
          (down_proj): Linear(in_features=1536, out_features=576, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((576,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((576,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm

In [8]:
from peft import LoraConfig, get_peft_model
lora_config = LoraConfig(
    r=64,
    lora_alpha=32,
    target_modules=['o_proj', 'q_proj', 'up_proj', 'v_proj', 'k_proj', 'down_proj', 'gate_proj'],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

In [9]:
import transformers
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    dataset_text_field="text",
    peft_config=lora_config,
    max_seq_length=1500,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        max_steps=100,
        learning_rate=2e-4,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        save_strategy="epoch",
        report_to="tensorboard",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


Step,Training Loss
1,3.042000
2,2.811900
3,3.185900
4,3.120700
5,3.241400
6,2.880000
7,3.057900
8,3.017800
9,3.247500
10,2.979700


TrainOutput(global_step=100, training_loss=2.6121045756340027, metrics={'train_runtime': 102.9408, 'train_samples_per_second': 3.886, 'train_steps_per_second': 0.971, 'total_flos': 89839698324480.0, 'train_loss': 2.6121045756340027, 'epoch': 8.0})

In [10]:
from transformers import pipeline
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'Mamba2ForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausal

## Inference

In [11]:
prompt = """
Below is an instruction that extracts keywords from a paragraph. Write keywords for the paragraph described below.

### Instruction:
Paragraph:
In this work we propose RELDEC, a novel approach for sequential decoding of moderate length low-density parity-check (LDPC) codes. The main idea behind RELDEC is that an optimized decoding policy is subsequently obtained via reinforcement learning based on a Markov decision process (MDP). In contrast to our previous work, where an agent learns to schedule only a single check node (CN) within a group (cluster) of CNs per iteration, in this work we train the agent to schedule all CNs in a cluster, and all clusters in every iteration. That is, in each learning step of RELDEC an agent learns to schedule CN clusters sequentially depending on a reward associated with the outcome of scheduling a particular cluster. We also modify the state space representation of the MDP, enabling RELDEC to be suitable for larger block length LDPC codes than those studied in our previous work. Furthermore, to address decoding under varying channel conditions, we propose two related schemes, namely, agile meta-RELDEC (AM-RELDEC) and meta-RELDEC (M-RELDEC), both of which employ meta-reinforcement learning. The proposed RELDEC scheme significantly outperforms standard flooding and random sequential decoding for a variety of LDPC codes, including codes designed for 5G new radio.

### Response:
"""

result = pipe(prompt, max_length=1500)
generated_text = result[0]['generated_text']
print(generated_text.split("### End")[0])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



Below is an instruction that extracts keywords from a paragraph. Write keywords for the paragraph described below.

### Instruction:
Paragraph:
In this work we propose RELDEC, a novel approach for sequential decoding of moderate length low-density parity-check (LDPC) codes. The main idea behind RELDEC is that an optimized decoding policy is subsequently obtained via reinforcement learning based on a Markov decision process (MDP). In contrast to our previous work, where an agent learns to schedule only a single check node (CN) within a group (cluster) of CNs per iteration, in this work we train the agent to schedule all CNs in a cluster, and all clusters in every iteration. That is, in each learning step of RELDEC an agent learns to schedule CN clusters sequentially depending on a reward associated with the outcome of scheduling a particular cluster. We also modify the state space representation of the MDP, enabling RELDEC to be suitable for larger block length LDPC codes than those st

In [12]:
prompt = """
Below is an instruction that extracts keywords from a paragraph. Write keywords for the paragraph described below.

### Instruction:
Paragraph:
This research presents an Attention-aware Deep Network for facial expression recognition in the presence of masks. Recognizing facial expression is a crucial problem in the current scenario due to the widespread use of masks due to the COVID-19 pandemic. The model utilizes attention mechanisms to focus on the relevant facial regions for expression recognition, while disregarding the regions occluded by masks. The model is trained and evaluated on a dataset of individuals wearing masks, and experimental results demonstrate its effectiveness in recognizing facial expressions accurately, even in the presence of masks. The proposed approach significantly improves the performance of expression recognition in comparison to existing methods and shows the potential of attention mechanisms in addressing the challenges of facial expression recognition with masks. Datasets are categorized into 7 Classes which represent 7 human expressions including happy, sad, fear, angry, disgust, neutral and surprised.

### Response:
"""


result = pipe(prompt, max_length=1500)
generated_text = result[0]['generated_text']
print(generated_text.split("### End")[0])


Below is an instruction that extracts keywords from a paragraph. Write keywords for the paragraph described below.

### Instruction:
Paragraph:
This research presents an Attention-aware Deep Network for facial expression recognition in the presence of masks. Recognizing facial expression is a crucial problem in the current scenario due to the widespread use of masks due to the COVID-19 pandemic. The model utilizes attention mechanisms to focus on the relevant facial regions for expression recognition, while disregarding the regions occluded by masks. The model is trained and evaluated on a dataset of individuals wearing masks, and experimental results demonstrate its effectiveness in recognizing facial expressions accurately, even in the presence of masks. The proposed approach significantly improves the performance of expression recognition in comparison to existing methods and shows the potential of attention mechanisms in addressing the challenges of facial expression recognition w